# Women in Data Science 2022

# Notebook set up

## Imports

In [1]:
import random, os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import BayesianRidge, Ridge
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
from sklearn.model_selection import (
    RandomizedSearchCV,
    train_test_split,
    cross_validate,
    cross_val_score,
    LeaveOneGroupOut,
)
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    PowerTransformer,
    StandardScaler,
)

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

## Global notebook setup

### Global variables

In [2]:
TRAIN_PATH = "../input/widsdatathon2022/train.csv"
TEST_PATH = "../input/widsdatathon2022/test.csv"
SUBMISSION_PATH = "../output/"
TARGET = "site_eui"
SCORE = "neg_root_mean_squared_error"
SEED = 777

### Display options

In [3]:
pd.options.display.max_rows = 100

# Data preprocessing

## Read in data

In [4]:
def read_data(train_path, test_path):
    """Reads in train and test data."""
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    return train_df, test_df

In [5]:
train, test = read_data(TRAIN_PATH, TEST_PATH)

In [6]:
train.shape

(75757, 64)

In [7]:
test.shape

(9705, 63)

## Duplicated data

In [8]:
def get_duplicates(df, drop_cols=None):
    """Returns duplicated values in a dataframe."""
    if drop_cols is not None:
        return df[df.drop(columns=drop_cols).duplicated()]
    else:
        return df[df.duplicated()]

In [9]:
train_duplicates = get_duplicates(train, ["id"])
test_duplicates = get_duplicates(test, ["id"])

In [10]:
train_duplicates.shape

(39, 64)

In [11]:
test_duplicates.shape

(5, 63)

In [12]:
def remove_duplicates(df, drop_cols=None):
    """Removes duplicated values in a dataframe"""
    df_clean = df.copy()
    if drop_cols is not None:
        df_clean = df_clean[~df_clean.drop(columns=drop_cols).duplicated()]
    else:
        df_clean = df_clean[~df_clean.duplicated()]
    return df_clean.reset_index(drop=True)

In [13]:
train = remove_duplicates(train, ["id"])

In [14]:
train.shape

(75718, 64)

## Missing data

In [15]:
def count_missing(df):
    """Counts the missing values in a dataframe"""
    missing_df = pd.DataFrame(
        df.isna().sum().sort_values(ascending=False), columns=["count"]
    )
    missing_df["percent"] = missing_df["count"] / df.shape[0]
    return missing_df.query("count != 0")

In [16]:
count_missing(train)

,count,percent
days_with_fog,45783,0.604651
direction_peak_wind_speed,41798,0.552022
max_wind_speed,41070,0.542407
direction_max_wind_speed,41070,0.542407
energy_star_rating,26682,0.352386
year_built,1817,0.023997


In [17]:
count_missing(test)

,count,percent
days_with_fog,9117,0.939413
max_wind_speed,8575,0.883565
direction_peak_wind_speed,8575,0.883565
direction_max_wind_speed,8575,0.883565
energy_star_rating,2254,0.232251
year_built,92,0.009480


## Data imputation: `energy_star`, `year_built`

In [18]:
def iterative_impute(train_df, test_df, model, ct, target, feat_names, seed):
    """Iteratively impute missing values with a desired model"""
    train_imp = ct.fit_transform(train_df.drop(columns=[target]))
    test_imp = ct.transform(test_df)

    imputer = IterativeImputer(estimator=model, random_state=seed)

    # TODO: fix feat_names with appropriate sklearn method
    cols = (
        ct.named_transformers_["onehotencoder"].get_feature_names().tolist()
        + feat_names
    )

    train_imp = pd.DataFrame(imputer.fit_transform(train_imp), columns=cols)
    test_imp = pd.DataFrame(imputer.transform(test_imp), columns=cols)

    return train_imp, test_imp

In [19]:
def replace_columns(df, df_imp, columns):
    """Replace columns in one dataframe with columns from another"""
    df_replaced = df.copy()

    for col in columns:
        df_replaced[col] = df_imp[col]

    return df_replaced

In [20]:
cat_imp = [
    'Year_Factor', 'State_Factor', 'building_class', 'facility_type'
]

num_imp = [
    'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION',
    'cooling_degree_days', 'heating_degree_days', 'precipitation_inches',
    'snowfall_inches', 'snowdepth_inches', 'avg_temp'
]

drop_imp = [
    'january_min_temp', 'january_avg_temp', 'january_max_temp',
    'february_min_temp', 'february_avg_temp', 'february_max_temp',
    'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp',
    'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp',
    'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp',
    'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp',
    'august_avg_temp', 'august_max_temp', 'september_min_temp',
    'september_avg_temp', 'september_max_temp', 'october_min_temp',
    'october_avg_temp', 'october_max_temp', 'november_min_temp',
    'november_avg_temp', 'november_max_temp', 'december_min_temp',
    'december_avg_temp', 'december_max_temp', 'days_below_30F', 'days_below_20F',
    'days_below_10F', 'days_below_0F', 'days_above_80F', 'days_above_90F',
    'days_above_100F', 'days_above_110F', 'direction_max_wind_speed',
    'direction_peak_wind_speed', 'max_wind_speed', 'days_with_fog',
    'id'
]

In [21]:
ct_imp = make_column_transformer(
    (OneHotEncoder(handle_unknown="ignore", sparse=False), cat_imp),
    ("passthrough", num_imp),
    ("drop", drop_imp)
)

# change to better model later
model = Ridge()

train_imp, test_imp = iterative_impute(train, test, model, ct_imp, TARGET, num_imp, SEED)

In [22]:
train = replace_columns(train, train_imp, ["energy_star_rating", "year_built"])
test = replace_columns(test, test_imp, ["energy_star_rating", "year_built"])

In [23]:
count_missing(train)

,count,percent
days_with_fog,45783,0.604651
direction_peak_wind_speed,41798,0.552022
max_wind_speed,41070,0.542407
direction_max_wind_speed,41070,0.542407


In [24]:
count_missing(test)

,count,percent
days_with_fog,9117,0.939413
max_wind_speed,8575,0.883565
direction_peak_wind_speed,8575,0.883565
direction_max_wind_speed,8575,0.883565


# Feature Engineering

## New Features: Entire Dataset

In [25]:
def has_feature(df, feat):
    """Adds a boolean feature if a feature exists or now"""
    return df[feat].notna().astype(int)

In [26]:
def feature_engineer(train_df, test_df):
    """Feature engineer the wids 2022 dataset"""
    train_df_eng = train_df.copy()
    test_df_eng = test_df.copy()
    
    # whether or not a building has a fog detector
    train_df_eng["has_fog_detector"] = has_feature(train_df_eng, "days_with_fog")
    test_df_eng["has_fog_detector"] = has_feature(test_df_eng, "days_with_fog")
    
    # whether or not a building has a wind detector
    train_df_eng["has_wind_detector"] = has_feature(train_df_eng, "max_wind_speed")
    test_df_eng["has_wind_detector"] = has_feature(test_df_eng, "max_wind_speed")
    
    return train_df_eng, test_df_eng

In [27]:
# TODO: Add more features
def add_cluster_labels(model, ct, train_df, test_df, target):
    """Adds cluster labels as a feature"""
    train_df_cluster = train_df.copy()
    test_df_cluster = test_df.copy()

    X_cluster = ct.fit_transform(train_df.drop(columns=[target]))
    test_cluster = ct.transform(test_df)

    model.fit(X_cluster)

    train_df_cluster["cluster_label"] = model.labels_
    test_df_cluster["cluster_label"] = model.predict(test_cluster)

    return train_df_cluster, test_df_cluster

In [28]:
train_df_eng, test_df_eng = feature_engineer(train, test)

## Cluster buildings

In [29]:
def plot_elbow_curve(X, model, k):
    visualizer = KElbowVisualizer(model, k=k)
    visualizer.fit(X)
    visualizer.show()
    return visualizer

In [30]:
def plot_silhouette(X, model):
    visualizer = SilhouetteVisualizer(model, colors="yellowbrick")
    visualizer.fit(X)
    visualizer.show()
    return visualizer

In [31]:
def add_cluster_labels(model, ct, train_df, test_df, target):
    """Adds cluster labels as a feature"""
    train_df_cluster = train_df.copy()
    test_df_cluster = test_df.copy()

    X_cluster = ct.fit_transform(train_df.drop(columns=[target]))
    test_cluster = ct.transform(test_df)

    model.fit(X_cluster)

    train_df_cluster["cluster_label"] = model.labels_
    test_df_cluster["cluster_label"] = model.predict(test_cluster)

    return train_df_cluster, test_df_cluster

In [32]:
cat_cluster = [
    'building_class', 'facility_type'
]

num_cluster = [
    'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION',
    'cooling_degree_days', 'heating_degree_days', 'precipitation_inches',
    'snowfall_inches', 'snowdepth_inches', 'avg_temp'
]

passthrough_cluster = [
    'has_fog_detector', 'has_wind_detector'
]

drop_cluster = [
    'Year_Factor', 'State_Factor', 'january_min_temp', 'january_avg_temp', 'january_max_temp',
    'february_min_temp', 'february_avg_temp', 'february_max_temp',
    'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp',
    'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp',
    'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp',
    'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp',
    'august_avg_temp', 'august_max_temp', 'september_min_temp',
    'september_avg_temp', 'september_max_temp', 'october_min_temp',
    'october_avg_temp', 'october_max_temp', 'november_min_temp',
    'november_avg_temp', 'november_max_temp', 'december_min_temp',
    'december_avg_temp', 'december_max_temp', 'days_below_30F', 'days_below_20F',
    'days_below_10F', 'days_below_0F', 'days_above_80F', 'days_above_90F',
    'days_above_100F', 'days_above_110F', 'direction_max_wind_speed',
    'direction_peak_wind_speed', 'max_wind_speed', 'days_with_fog',
    'id'
]

In [33]:
ct_cluster = make_column_transformer(
    (OneHotEncoder(handle_unknown="ignore"), cat_cluster),
    (StandardScaler(), num_cluster),
    ("passthrough", passthrough_cluster),
    ("drop", drop_cluster)
)

In [34]:
X_cluster = ct_cluster.fit_transform(train_df_eng.drop(columns=[TARGET]))

In [35]:
# k = (5, 20)
# model = KMeans()
# visualizer_elbow = plot_elbow_curve(X_cluster, model, k)

In [36]:
kmeans = KMeans(n_clusters=11, random_state=42)

In [37]:
# plot_silhouette(X_cluster, kmeans)

In [38]:
train_df_eng, test_df_eng = add_cluster_labels(
    kmeans,
    ct_cluster,
    train_df_eng,
    test_df_eng,
    TARGET
)

## New Features: Individual clusters

In [39]:
# TODO

# Modelling

## Create `X` and `y` datasets

In [40]:
def split_data(df, column, name):
    """Creates separate dataframes split on a single columns values"""
    dfs = {}

    for i in np.sort(df[column].unique()):
        split_df = df[df[column] == i]
        dfs[f"{name}_{i}"] = split_df

    return dfs

In [41]:
def create_X_y(dfs, target):
    """Separates train dfs into X and y datasets"""
    X_dfs = {}
    y_dfs = {}

    for name, df in dfs.items():
        X_dfs[name] = df.drop(columns=target)
        y_dfs[name] = df[target]

    return X_dfs, y_dfs

In [42]:
train_eng_dfs = split_data(train_df_eng, "cluster_label", "cluster")

In [43]:
X_trains, y_trains = create_X_y(train_eng_dfs, TARGET)
X_tests = split_data(test_df_eng, "cluster_label", "cluster")

## Column transformer

In [44]:
cat = [
    'building_class', 'facility_type'
]

num = [
    'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION',
    'cooling_degree_days', 'heating_degree_days', 'precipitation_inches',
    'snowfall_inches', 'snowdepth_inches', 'avg_temp'
]

passthrough = [
    'has_fog_detector', 'has_wind_detector'
]

drop = [
    'Year_Factor', 'State_Factor', 'january_min_temp', 'january_avg_temp', 'january_max_temp',
    'february_min_temp', 'february_avg_temp', 'february_max_temp',
    'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp',
    'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp',
    'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp',
    'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp',
    'august_avg_temp', 'august_max_temp', 'september_min_temp',
    'september_avg_temp', 'september_max_temp', 'october_min_temp',
    'october_avg_temp', 'october_max_temp', 'november_min_temp',
    'november_avg_temp', 'november_max_temp', 'december_min_temp',
    'december_avg_temp', 'december_max_temp', 'days_below_30F', 'days_below_20F',
    'days_below_10F', 'days_below_0F', 'days_above_80F', 'days_above_90F',
    'days_above_100F', 'days_above_110F', 'direction_max_wind_speed',
    'direction_peak_wind_speed', 'max_wind_speed', 'days_with_fog',
    'id', 'cluster_label'
]

In [45]:
ct = make_column_transformer(
    (OneHotEncoder(handle_unknown="ignore"), cat),
    ("passthrough", num + passthrough),
    ("drop", drop)
)

## Cross validation

In [46]:
# not in utils yet, need to finalize
def cross_validate_all(model, X_trains, y_trains, ct, scoring, cv=5, n_jobs=-1):
    
    scores = {}
    
    for (name1, X_train), (name2, y_train) in zip(X_trains.items(), y_trains.items()):
        assert name1 == name2
        pipe = make_pipeline(
            ct,
            model
        )
        scores[name1] = np.mean(cross_validate(pipe, X_train, y_train, scoring=scoring, cv=cv, n_jobs=n_jobs)['test_score'])
    
    return scores

In [47]:
regressor = lgb.LGBMRegressor(force_row_wise=True)

In [48]:
scores = cross_validate_all(regressor, X_trains, y_trains, ct, SCORE)

In [49]:
scores

{'cluster_0': -49.45509507997053,
 'cluster_1': -33.30829191395533,
 'cluster_2': -40.97358315843623,
 'cluster_3': -41.69027308975572,
 'cluster_4': -59.04436712139942,
 'cluster_5': -34.968685587726895,
 'cluster_6': -38.590760975728855,
 'cluster_7': -47.312686405746575,
 'cluster_8': -60.22807679023938,
 'cluster_9': -57.972726037251206,
 'cluster_10': -46.57478448029247}

## Hyperparameter Tuning

In [50]:
# TODO

## Ensembling

In [51]:
# TODO

## Final models

In [52]:
# not in utils yet, need to finalize
def train_final_models(model, X_trains, y_trains, ct):
    models = {}
    
    for (name1, X_train), (name2, y_train) in zip(X_trains.items(), y_trains.items()):
        assert name1 == name2
        pipe = make_pipeline(
            ct,
            model
        )

        models[name1] = pipe.fit(X_train, y_train)
    
    return models

In [53]:
regressor = lgb.LGBMRegressor(force_row_wise=True)

In [54]:
models = train_final_models(regressor, X_trains, y_trains, ct)

# Predictions

In [55]:
# not in utils yet, need to finalize
def get_all_predictions(models, X_tests, target):
    predictions = pd.DataFrame()
    
    for name, X_test in X_tests.items():
        pred = {
            "id": X_test["id"],
            target: models[name].predict(X_test)
        }
        
        predictions = pd.concat([predictions, pd.DataFrame(pred)])
        
    predictions = predictions.sort_values(by="id")
    
    return predictions

In [56]:
predictions = get_all_predictions(models, X_tests, TARGET)

In [57]:
predictions.to_csv("submission.csv", index=False)